In [1]:
import pandas as pd

from twitter_scraper import settings
from twitter_scraper.utils import fileio

stop_words_eng = fileio.read_content(settings.STOP_WORDS_ENG, 'json')
stop_words_hrv = fileio.read_content(settings.STOP_WORDS_HRV, 'json')
stop_words = stop_words_eng + stop_words_hrv 

tweets_df = pd.read_csv(settings.TWEETS_CSV.replace('tweets.csv', 'tweets-text.csv'), low_memory=False)
tweets_df['stemmed'] = tweets_df['stemmed'].map(eval)
tweets_df['clean_stem'] = tweets_df['stemmed'].transform(lambda x: list(filter(lambda y: y is not None and y not in stop_words, x)))

[WARNING] 2022-11-06 07:31:15 discord.client - PyNaCl is not installed, voice will NOT be supported


In [2]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3241556 entries, 0 to 3241555
Data columns (total 24 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   id                        int64  
 1   user_id                   int64  
 2   user_id_str               int64  
 3   full_text                 object 
 4   created_at                object 
 5   hashtags                  object 
 6   user_mentions             object 
 7   retweet_from_user_id      float64
 8   retweet_from_user_id_str  object 
 9   retweet_from_tweet_id     float64
 10  in_reply_to_status_id     float64
 11  in_reply_to_user_id       float64
 12  geo                       object 
 13  coordinates               object 
 14  retweet_count             int64  
 15  favorite_count            int64  
 16  lang                      object 
 17  langid                    object 
 18  week                      object 
 19  month                     object 
 20  is_covid                

In [3]:
gdf = tweets_df[['langid', 'clean_stem']][tweets_df['langid'].isin(('en', 'hr', 'bs', 'sr', 'sl'))].sample(100000).groupby('langid').sum()
a = {key: list(set(val['clean_stem'])) for key, val in gdf.to_dict(orient='index').items()}

In [6]:
a = {key: sorted(val) for key, val in a.items()}

In [7]:
fileio.write_content(settings.INPUT_DIR + '/all_words.json', a, 'json')

In [8]:
import csv
tweets_df.to_csv(settings.TWEETS_CSV.replace('tweets.csv', 'tweets-text-processed.csv'), encoding='utf-8', index=False, quoting=csv.QUOTE_ALL)